In [ ]:
import torch
import torch.nn as nn
class conv_block(nn.Module):
  def __init__(self,in_channels,out_channels,**kwargs):
    super(conv_block,self).__init__()
    self.relu=nn.ReLU()
    self.conv1=nn.Conv2d(in_channels,out_channels,**kwargs)
    self.bn1=nn.BatchNorm2d(out_channels)

  def forward(self,x):
    return self.relu(self.bn1(self.conv1(x)))

x=torch.rand(1,3,224,224)
layer=conv_block(3,10)
out=layer(x)
print(out.shape)


TypeError: ignored

In [ ]:
import torch
import torch.nn as nn
class conv_block(nn.Module):
  def __init__(self,in_channels,out_channels,**kwargs):
    super(conv_block,self).__init__()
    self.relu=nn.ReLU()
    self.conv1=nn.Conv2d(in_channels,out_channels,**kwargs)
    self.bn1=nn.BatchNorm2d(out_channels)

  def forward(self,x):
    return self.relu(self.bn1(self.conv1(x)))


In [ ]:
class Inceptionblock(nn.Module):
  def __init__(self,in_channels,out_1x1,red_3x3,out_3x3,red_5x5,out_5x5,out_1x1pool):
    super(Inceptionblock,self).__init__()
    self.branch1=conv_block(in_channels,out_1x1,kernel_size=1)
    
    self.branch2=nn.Sequential(conv_block(in_channels,red_3x3,kernel_size=1),
                               conv_block(red_3x3,out_3x3,kernel_size=3,padding=1))
    
    self.branch3=nn.Sequential(conv_block(in_channels,red_5x5,kernel_size=1),
                              conv_block(red_5x5,out_5x5,kernel_size=5,padding=2))
    
    self.branch4=nn.Sequential(nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
                               conv_block(in_channels,out_1x1,kernel_size=1))
    
  def forward(self,x):
    return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x),self.branch4(x)],1)
layerI=Inceptionblock(192,64,96,128,16,32,32)
x=torch.rand(1,192,224,224)
out=layerI(x)
print(out.shape)

torch.Size([1, 288, 224, 224])


In [ ]:
class GoogleNet(nn.Module):
  def __init__(self, in_channels,n_classes):
    super(GoogleNet,self).__init__()
    self.conv1=conv_block(in_channels,64,kernel_size=(7,7),stride=(2,2),padding=(3,3))
    self.maxpool1=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    self.conv2=conv_block(64,192,kernel_size=3,stride=1,padding=1)
    self.maxpool2=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
  
    self.inception3a=Inceptionblock(192,64,96,128,16,32,32)
    self.inception3b=Inceptionblock(256,128,128,192,32,96,64)
    self.maxpool3=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    self.inception4a=Inceptionblock(480,192,96,208,16,48,64)
    self.inception4b=Inceptionblock(512,160,112,224,24,64,64)
    self.inception4c=Inceptionblock(512,128,128,256,24,64,64)
    self.inception4d=Inceptionblock(512,112,144,288,32,64,64)
    self.inception4e=Inceptionblock(528,256,160,320,32,128,128)
    self.maxpool4=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

    self.inception5a=Inceptionblock(832,256,160,320,32,128,128)
    self.inception5b=Inceptionblock(832,384,192,384,48,128,128)
    self.maxpool5=nn.AvgPool2d(kernel_size=7,stride=1)
    self.drpoout=nn.Dropout2d(p=.5)
    self.fc=nn.Linear(1024,n_classes)
  def forward(self,x):
    x=self.conv1(x)
    x=self.maxpool1(x)
    x=self.conv2(x)
    x=self.maxpool2(x)
    x=self.inception3a(x)
    print(x.shape)
    x=self.inception3b(x)
    x=self.maxpool3(x)
    x=self.inception4a(x)
    x=self.inception4b(x)
    x=self.inception4c(x)
    x=self.inception4d(x)
    x=self.inception4e(x)
    x=self.maxpool4(x)
    x=self.inception5a(x)
    x=self.inception5b(x)
    x=self.maxpool5(x)
    x=self.drpoout(x)
    x=x.reshape(x.shape[0],-1)
    x=self.fc(x)
    return x
    
model=GoogleNet(3,1000)

x=torch.rand(1,3,224,224)
out=model(x)
print(out.shape)


torch.Size([1, 288, 28, 28])


RuntimeError: ignored

In [ ]:
# navie Inception module
import torch
import torch.nn as nn
class inception_navie(nn.Module):
  def __init__(self,in_channels,out_1x1,out_3x3,out_5x5):
    super(inception_navie,self).__init__()
    self.branch1=nn.Conv2d(in_channels,out_1x1,kernel_size=1)
    self.branch2=nn.Conv2d(in_channels,out_3x3,kernel_size=3,stride=1,padding=1)
    self.branch3=nn.Conv2d(in_channels,out_5x5,kernel_size=5,stride=1,padding=2)
    self.branch4=nn.MaxPool2d(kernel_size=3,stride=1,padding=1)
  def forward(self,x):
    return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x),self.branch4(x)],1)

model=inception_navie(20,20,20,20)
inp=torch.rand(1,20,30,30)
out=model(inp)
print(out.shape)
#print(model)


torch.Size([1, 80, 30, 30])


In [ ]:
# modified Inception module insert 1x1 conv at each block
import torch
import torch.nn as nn
class inception_navie(nn.Module):
  def __init__(self,in_channels,out_1x1,out_3x3r,out_3x3,out_5x5r,out_5x5):
    super(inception_navie,self).__init__()
    self.branch1=nn.Conv2d(in_channels,out_1x1,kernel_size=1)
    self.branch2=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=3,stride=1,padding=1))
    
    self.branch3=nn.Sequential(nn.Conv2d(in_channels,out_5x5r,kernel_size=1),
                               nn.Conv2d(out_5x5r,out_5x5,kernel_size=5,stride=1,padding=2))
    
    self.branch4=nn.Sequential(nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
                               nn.Conv2d(in_channels,out_1x1,kernel_size=1))
  def forward(self,x):
    return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x),self.branch4(x)],1)

model=inception_navie(20,20,60,40,50,50)
inp=torch.rand(1,20,30,30)
out=model(inp)
print(out.shape)
#print(model)

torch.Size([1, 130, 30, 30])


In [ ]:
# Inception A module
import torch
import torch.nn as nn
class inception_navie(nn.Module):
  def __init__(self,in_channels,out_1x1,out_3x3r,out_3x3,out_5x5r,out_5x5):
    super(inception_navie,self).__init__()
    self.branch1=nn.Conv2d(in_channels,out_1x1,kernel_size=1)
    self.branch2=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=3,stride=1,padding=1))
    
    self.branch3=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=3,stride=1,padding=1),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=3,stride=1,padding=1))
    
    self.branch4=nn.Sequential(nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
                               nn.Conv2d(in_channels,out_1x1,kernel_size=1))
  def forward(self,x):
    return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x),self.branch4(x)],1)

model=inception_navie(20,20,60,40,50,50)
inp=torch.rand(1,20,30,30)
out=model(inp)
print(out.shape)
#print(model)

torch.Size([1, 120, 30, 30])


In [ ]:
# Inception B module
import torch
import torch.nn as nn
class inception_navie(nn.Module):
  def __init__(self,in_channels,out_1x1,out_3x3r,out_3x3,out_5x5r,out_5x5):
    super(inception_navie,self).__init__()
    self.branch1=nn.Conv2d(in_channels,out_1x1,kernel_size=1)
    self.branch2=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,3),stride=1,padding=(0,1)),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=(3,1),stride=1,padding=(1,0)))
    
    self.branch3=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=3,stride=1,padding=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,3),stride=1,padding=(0,1)),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=(3,1),stride=1,padding=(1,0)))
    
    self.branch4=nn.Sequential(nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
                               nn.Conv2d(in_channels,out_1x1,kernel_size=1))
  def forward(self,x):
    return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x),self.branch4(x)],1)

model=inception_navie(20,20,60,40,50,50)
inp=torch.rand(1,20,30,30)
out=model(inp)
print(out.shape)
#print(model)

torch.Size([1, 120, 30, 30])


In [ ]:
# Inception C module
import torch
import torch.nn as nn
class inception_navie(nn.Module):
  def __init__(self,in_channels,out_1x1,out_3x3r,out_3x3,out_5x5r,out_5x5):
    super(inception_navie,self).__init__()
    self.branch1=nn.Conv2d(in_channels,out_1x1,kernel_size=1)
    self.branch2=nn.Sequential(nn.AvgPool2d(kernel_size=3,stride=1,padding=1),
                               nn.Conv2d(in_channels,out_1x1,kernel_size=1))
    
    self.branch3=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,7),stride=1,padding=(1,2)),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=(7,1),stride=1,padding=(2,1)))
    
    self.branch4=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,7),stride=1,padding=(1,2)),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(7,1),stride=1,padding=(2,1)),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,7),stride=1,padding=(1,2)),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=(7,1),stride=1,padding=(2,1)))
  def forward(self,x):
    return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x),self.branch4(x)],1)

model=inception_navie(20,20,60,40,50,50)
inp=torch.rand(1,20,30,30)
out=model(inp)
print(out.shape)
#print(model)

torch.Size([1, 120, 30, 30])


In [ ]:
 # pool of square window of size=3, stride=2
m = nn.AvgPool2d(3, stride=1,padding=1)
# pool of non-square window
#m = nn.AvgPool2d((3, 2), stride=(2, 1))
input = torch.randn(1, 20, 30, 30)
output = m(input)
print(output.shape)

torch.Size([1, 20, 30, 30])


In [ ]:
branch4=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,7),stride=1,padding=(1,2)),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(7,1),stride=1,padding=(2,1)),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,7),stride=1,padding=(1,2)),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=(7,1),stride=1,padding=(2,1)))

input = torch.randn(1, 20, 30, 30)
output = branch4(input)
print(output.shape)

torch.Size([1, 40, 30, 30])


In [ ]:
branch3=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,7),stride=1,padding=(1,2)),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=(7,1),stride=1,padding=(2,1)))

input = torch.randn(1, 20, 30, 30)
output = branch3(input)
print(output.shape)

torch.Size([1, 40, 30, 30])


In [ ]:
out_1x1=20
branch2=nn.Sequential(nn.AvgPool2d(kernel_size=3,stride=1,padding=1))

input = torch.randn(1, 20, 30, 30)
output = branch2(input)
print(output.shape)

torch.Size([1, 20, 30, 30])


In [ ]:
# Reduction block A
import torch
import torch.nn as nn
class inception_navie(nn.Module):
  def __init__(self,in_channels,out_1x1,out_3x3r,out_3x3,out_5x5r,out_5x5):
    super(inception_navie,self).__init__()
    self.branch1=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=3,stride=1,padding=1),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=3,stride=1,padding=1))
    
    self.branch2=nn.Conv2d(in_channels,out_3x3,kernel_size=3,stride=1,padding=1)
    
    self.branch3=nn.MaxPool2d(kernel_size=3,stride=1,padding=1)
  def forward(self,x):
    return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x)],1)

model=inception_navie(20,20,60,40,50,50)
inp=torch.rand(1,20,30,30)
out=model(inp)
print(out.shape)
#print(model)

torch.Size([1, 100, 30, 30])


In [ ]:
# Reduction block B
import torch
import torch.nn as nn
class inception_navie(nn.Module):
  def __init__(self,in_channels,out_1x1,out_3x3r,out_3x3,out_5x5r,out_5x5):
    super(inception_navie,self).__init__()
    self.branch1=nn.Sequential(nn.Conv2d(in_channels,out_3x3r,kernel_size=1),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(1,7),stride=1,padding=(1,2)),
                               nn.Conv2d(out_3x3r,out_3x3r,kernel_size=(7,1),stride=1,padding=(2,1)),
                               nn.Conv2d(out_3x3r,out_3x3,kernel_size=3,stride=1,padding=1))
    
    self.branch2=nn.Sequential(nn.Conv2d(in_channels,out_1x1,kernel_size=1),
                               nn.Conv2d(out_1x1,out_3x3,kernel_size=3,stride=1,padding=1))
    
    self.branch3=nn.MaxPool2d(kernel_size=3,stride=1,padding=1)
  def forward(self,x):
    return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x)],1)

model=inception_navie(20,20,60,40,50,50)
inp=torch.rand(1,20,30,30)
out=model(inp)
print(out.shape)
#print(model)

torch.Size([1, 100, 30, 30])
